In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/update/update_bm_revision.json"

In [ ]:
from collections import defaultdict
runs = defaultdict(list)

BMS = get_all_runs(RESULT_JSON)
IN_PLACE = 'in_place'
COPY_WRITE = 'get_update'

for bm in BMS:
    if IN_PLACE in bm['name']:
        runs[IN_PLACE].append(bm)
    elif COPY_WRITE in bm['name']:
        runs[COPY_WRITE].append(bm)
    else:
        raise RuntimeError(f"Unknown benchmark type {bm['name']}")

runs[IN_PLACE].sort(key=lambda x: x['threads'])
runs[COPY_WRITE].sort(key=lambda x: x['threads'])

# pprint(runs)

In [ ]:
in_place_w_threads = [(run['threads'], run['items_per_second'] / MILLION) for run in runs[IN_PLACE]]
copy_write = [run['items_per_second'] / MILLION for run in runs[COPY_WRITE]]

threads, in_place = zip(*in_place_w_threads)

fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot(threads, in_place, label="In-Place", ls='-', color=PMEM_COLOR, lw=3)
ax.plot(threads, copy_write, label="CoW", ls='--', color=DRAM_COLOR, lw=3)
ax.set_yticks(range(0, 16, 5))

ax.set_ylabel("Throughput\n(Mops/s)", fontsize=18)
ax.set_xlabel("# Threads", fontsize=18)
ax.legend(frameon=False, loc='upper left', fontsize=18,
          columnspacing=1.3, handletextpad=0.3, labelspacing=0.1,
         bbox_to_anchor=(-0.05, 1.25))

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(18)

# ax.yaxis.set_label_coords(-0.12, 0.45)

ax.set_axisbelow(True)
ax.grid(axis='y', which='major')
hide_border(ax)
plt.tight_layout()
fig.savefig('charts/update.pdf', bbox_inches='tight')
fig.savefig('charts/update.svg', bbox_inches='tight')